# <center>Learning how to play Space Invaders with RL.</center>

![space invaders poster image](https://cdn-images-1.medium.com/max/1200/1*Ya6drllhmC82yh0J-elrPA.jpeg)

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3803355718751304384
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3278045184
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13406098486108312404
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
import gym
import numpy as np
from dqn.model import ModelVanilla
from dqn.train import Train 
from dqn.dqnAgent import DQNAgent
import matplotlib.pyplot as plt

from gym.wrappers.atari_preprocessing import AtariPreprocessing
from baselines.common.atari_wrappers import NoopResetEnv, MaxAndSkipEnv, wrap_deepmind

Using TensorFlow backend.


In [11]:
env = gym.make('CartPole-v1')

#env = NoopResetEnv(env, noop_max=30)
#env = wrap_deepmind(env, episode_life=False, clip_rewards=False, frame_stack=True, scale=False)

**Observation space**: the observation space consists on 3 sequential images of size 210x160

**Action space**: there are 6 possible actions: *left, right, shoot, left-shoot, right-shoot, do nothing*.

In [12]:
print(env.observation_space.shape)
print(env.action_space)

(4,)
Discrete(2)


### Agent acting randomly

In [14]:
from tqdm import tqdm

reward_random = []

for i in tqdm(range(500)):
    total_reward = 0
    env.reset()
    while True:
        #env.render()
        next_state, reward, done, info = env.step((env.action_space.sample())) # select random action
        total_reward += reward
        if done:
            reward_random.append(total_reward)
            break

    env.close()
    
print("Average score over a {} episodes: {}".format(500, np.asarray(reward_random).mean()))


100%|██████████| 500/500 [00:00<00:00, 5618.90it/s]

Average score over a 500 episodes: 21.944


### State sequence

Each state will look like a sequence of 4 frames. Every frame is cropped and preprocessed for faster and more efficient computations.

In [15]:
env.reset()

# Create a new state
next_state, reward, done, info = env.step((env.action_space.sample())) # select random action

print(np.asarray(next_state).transpose(2, 0, 1).shape)
# Plot the state
fig, axes = plt.subplots(1, 4, figsize=(12, 4))
ax = axes.ravel()

for i, frame in enumerate(np.asarray(next_state).transpose(2, 0, 1)):
    ax[i].imshow(frame)

ValueError: axes don't match array

### Establishing a benchmark

I'll be using two benchmarks:
* The performance of an agent without training over 100 games
* The performance of a human
    * My avg performance over 50 games: 520
    * Avg performance of profesional tester according to DeepMind's Paper: 1652

In [16]:
episodes_eval = 500
agent_bench = DQNAgent(action_size=4)
benchmark = Train(env, agent_bench)

# Evaluate performance
benchmark.train(episodes_eval, learn=False, render=False)

# Plot rewards
benchmark.plot_rewards()


  0%|          | 0/500 [00:00<?, ?it/s]

IndexError: tuple index out of range

## Training

In [5]:
episodes_train = 50000
agent_train = DQNAgent(dueling=False, action_size=4)
# Load existing saved weights
# Comment if don't want to load pre-trained weights
#agent_train.load_weights('best_model_dueling')

train = Train(env, agent_train)
# Train the agent
train.train(episodes_train, learn=True, render=False)

# Plot rewards
train.plot_rewards(mean_avg=10)
train.plot_rewards(mean_avg=50)
train.plot_rewards(mean_avg=100)

 score: 2.0:   1%|▏         | 658/50000 [11:11<20:04:42,  1.46s/it]     

KeyboardInterrupt: 

In [ ]:
# Evaluate the trained agent
# agent_trained = train.agent
agent_trained = DQNAgent(dueling=True)
agent_trained.load_weights('best_model_dueling')
evaluation = Train(env, agent_trained)

# Evaluate performance
evaluation.train(episodes_eval, learn=False, render=False, monitor=True)

# Plot rewards
evaluation.plot_rewards(mean_avg=10)

In [ ]:
# Evaluate the trained agent
# agent_trained = train.agent
agent_trained = DQNAgent(dueling=False)
agent_trained.load_weights('test_pong')
evaluation = Train(env, agent_trained)

# Evaluate performance
evaluation.train(episodes_eval, learn=False, render=False, monitor=True)

# Plot rewards
evaluation.plot_rewards(mean_avg=10)